# Quick Start Example (Optional)

This notebook demonstrates basic data access patterns using DuckDB and pandas. For a deeper dive into building full analytics agents on MotherDuck (prompt design, MCP integration, security), see [MotherDuck’s analytics agent guide](https://motherduck.com/docs/key-tasks/ai-and-motherduck/building-analytics-agents/).

## Contents
- Tool installation
- Loading Parquet files
- SQL queries
- Log file parsing
- Submission format


## Install Tools

Run this once:


In [1]:
# %pip install -q duckdb pandas pyarrow lancedb

import duckdb
import pandas as pd
import os
from pathlib import Path

print("Ready.")


Ready.


## Connect to DuckDB

DuckDB lets you run SQL queries on Parquet files. It's fast and works well for analytics.


In [2]:
con = duckdb.connect('retail.duckdb')
print("Connected to DuckDB")


Connected to DuckDB


## Option 1: Load Data into local DuckDB

Load Parquet files into DuckDB so you can query them:


In [3]:
# List all parquet files in our GCS bucket
files = con.execute("SELECT file FROM glob('gs://antm-dataset/**/*.parquet')").fetchall()

# Create table for each file
for file_path, in files:
    table_name = Path(file_path).stem
    con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM read_parquet('{file_path}')")

# Show tables
print(f"\nCreated {len(files)} tables")
con.execute("SHOW TABLES").fetchall()


Created 16 tables


[('catalog_sales',),
 ('customer',),
 ('customer_address',),
 ('customer_demographics',),
 ('date_dim',),
 ('household_demographics',),
 ('income_band',),
 ('inventory',),
 ('item',),
 ('logs',),
 ('promotion',),
 ('reason',),
 ('store',),
 ('store_returns',),
 ('store_sales',),
 ('warehouse',),
 ('web_sales',)]

## Option 2: Connect to MotherDuck

MotherDuck gives your local DuckDB cloud compute resources. It also lets you share data with others easily. 

1. Go to [app.motherduck.com](https://app.motherduck.com) and create an account.
2. [create an access token](https://motherduck.com/docs/key-tasks/authenticating-and-connecting-to-motherduck/authenticating-to-motherduck/#creating-an-access-token)


In [ ]:
os.environ["motherduck_token"] = "your_actual_token_here"

Create [read-write clone](https://motherduck.com/docs/key-tasks/ai-and-motherduck/building-analytics-agents/#read-write-access--sandboxing) of 'antm_hack' MotherDuck Share.

In [ ]:
if os.environ.get("motherduck_token") == "your_actual_token_here":
    print("Using local DuckDB")
else:
    print("Creating 'antm_hack_rw' database, from 'antm_hack' share")

    con.execute("CREATE DATABASE antm_hack_rw FROM 'md:_share/antm_hack/88329567-1b97-4593-9696-73fd2be9c63d'")
    con.execute("USE antm_hack_rw")

## Run a Query

Example: Find top customers by spending


In [ ]:
if customer_path.exists():
    query = """
    SELECT 
        c_customer_sk,
        c_first_name,
        c_last_name
    FROM customer
    ORDER BY c_customer_sk
    LIMIT 5
    """
    
    result = con.execute(query).df()
    print(result)


## Read Log Files

Logs are in JSONL format (one JSON object per line). You can use pandas:


In [7]:
ex_path = Path('../dataset/data/logs/customer_service.jsonl')

In [8]:
if ex_path.exists():
    logs = pd.read_json('../dataset/data/logs/customer_service.jsonl', lines=True)
    print(f"Loaded {len(logs)} events")
    print(logs.head())


Loaded 9400 events
  ticket_id                 timestamp        category  customer_sk  priority  \
0  CS-22048 2022-04-25 03:40:36+00:00  product_defect        30144    medium   
1  CS-79521 2020-08-18 23:21:33+00:00  return_request        53158    medium   
2  CS-70494 2019-09-10 08:57:17+00:00  product_defect        51657       low   
3  CS-58144 2018-12-21 00:27:03+00:00    out_of_stock        80348  critical   
4  CS-11925 2018-08-17 09:24:33+00:00  return_request        19650       low   

   item_sk  warehouse_sk request_id request_source status assigned_to  \
0   2114.0           2.0        NaN            NaN    NaN         NaN   
1   9741.0           4.0        NaN            NaN    NaN         NaN   
2   1428.0           NaN        NaN            NaN    NaN         NaN   
3   2067.0           3.0        NaN            NaN    NaN         NaN   
4   2105.0           1.0        NaN            NaN    NaN         NaN   

  requester_email notes completed_date  
0             NaN   

## Alternative: Query Logs with DuckDB

You can also load logs into DuckDB and use SQL:


In [9]:
if ex_path.exists():
    con.execute("CREATE OR REPLACE TABLE logs AS SELECT * FROM read_json_auto('../dataset/data/logs/customer_service.jsonl')")
    
    result = con.execute("""
        SELECT category, COUNT(*) as count 
        FROM logs 
        GROUP BY category
        ORDER BY count DESC
        LIMIT 10
    """).df()
    
    print(result)


          category  count
0   shipping_delay   2715
1     out_of_stock   2322
2   product_defect   1894
3   return_request   1382
4  general_inquiry    887
5             None    200


## Using LanceDB for Semantic Search

LanceDB is useful when you need to search by meaning rather than exact matches. Example use case: searching through log descriptions or PDF content.


In [10]:
import lancedb

# Connect to LanceDB
db = lancedb.connect('lance_db')

# Example: Load log data into LanceDB
if ex_path.exists():
    log_data = pd.read_json(ex_path, lines=True)
    
    # Create a table
    table = db.create_table("logs", data=log_data, mode="overwrite")
    
    print(f"Loaded {len(log_data)} rows into LanceDB")
    
    # Query example: Filter logs
    results = table.search().where("category = 'out_of_stock'").limit(5).to_pandas()
    print("\nSample product view events:")
    print(results)
else:
    print("Log files not found.")


## Query LanceDB table or pandas DataFrame with DuckDB
arrow_table = table.to_lance()
con.query("SELECT * FROM arrow_table")
con.query("SELECT * FROM results")


Loaded 9400 rows into LanceDB

Sample product view events:
  ticket_id                 timestamp      category  customer_sk  priority  \
0  CS-58144 2018-12-21 00:27:03+00:00  out_of_stock        80348  critical   
1  CS-33960 2020-07-02 13:26:06+00:00  out_of_stock        90539    medium   
2  CS-95195 2020-04-06 06:10:54+00:00  out_of_stock       100395       low   
3  CS-48994 2018-09-22 20:20:02+00:00  out_of_stock        28988    medium   
4  CS-38729 2022-04-30 17:24:01+00:00  out_of_stock        31062    medium   

   item_sk  warehouse_sk request_id request_source status assigned_to  \
0   2067.0           3.0       None           None   None        None   
1   8784.0           2.0       None           None   None        None   
2   1471.0           1.0       None           None   None        None   
3   2115.0           2.0       None           None   None        None   
4   1679.0           2.0       None           None   None        None   

  requester_email notes completed

┌───────────┬──────────────────────────┬──────────────┬─────────────┬──────────┬─────────┬──────────────┬────────────┬────────────────┬────────┬─────────────┬─────────────────┬───────┬────────────────┐
│ ticket_id │        timestamp         │   category   │ customer_sk │ priority │ item_sk │ warehouse_sk │ request_id │ request_source │ status │ assigned_to │ requester_email │ notes │ completed_date │
│  varchar  │ timestamp with time zone │   varchar    │    int64    │ varchar  │ double  │    double    │   int32    │     int32      │ int32  │    int32    │      int32      │ int32 │     int32      │
├───────────┼──────────────────────────┼──────────────┼─────────────┼──────────┼─────────┼──────────────┼────────────┼────────────────┼────────┼─────────────┼─────────────────┼───────┼────────────────┤
│ CS-58144  │ 2018-12-20 16:27:03-08   │ out_of_stock │       80348 │ critical │  2067.0 │          3.0 │       NULL │           NULL │   NULL │        NULL │            NULL │  NULL │        

## Submission Format

Save your answers in CSV format:

```csv
question_id,answer_type,answer_value,confidence,explanation
1,customer_id,12345,high,Top customer by revenue
1,total_spent,50000,high,Sum of net_paid
```

Create submissions programmatically:


In [11]:
submission = pd.DataFrame([
    {'question_id': 1, 'answer_type': 'customer_id', 'answer_value': 12345, 'confidence': 'high', 'explanation': 'Top customer by revenue'},
    {'question_id': 1, 'answer_type': 'total_spent', 'answer_value': 50000, 'confidence': 'high', 'explanation': 'Sum of net_paid'},
])

print(submission)

# To save:
# submission.to_csv('my_submission.csv', index=False)


   question_id  answer_type  answer_value confidence              explanation
0            1  customer_id         12345       high  Top customer by revenue
1            1  total_spent         50000       high          Sum of net_paid


## Competition Structure

**Training Round (12:30-2:00):** 25 questions with answers provided. Practice only, no submission required.

**Test Round (2:00-6:00):** 30 questions without answers. Submit by 6:00 PM. Worth 70% of final score.

**Holdout Round (6:00-7:30):** 20 secret questions. We run your system automatically. Worth 30% of final score.

That's the basics. Check the README for more details.


## Tool Use Cases

**DuckDB:** SQL queries on structured data (Parquet tables) and logs. Fast for aggregations, joins, filtering.

**LanceDB:** Semantic search when you need to find things by meaning, not exact matches. Good for searching PDFs or finding similar log entries.

**MotherDuck:** Cloud version of DuckDB. Useful for sharing data/queries with teammates or working with larger datasets.


## MCP Server Quickstart

1. **Pick a language/runtime.** MCP servers only need stdin/stdout plus JSON-RPC. Python (`mcp`), TypeScript (`@modelcontextprotocol/server`), or Go all work.
2. **Choose resources.** Decide what data you’ll expose (DuckDB tables, Parquet files, PDF search). Give each a stable URI so clients know how to reference them.
3. **Implement tools.** Each MCP tool wraps an action—run SQL, summarize a log window, fetch a PDF section. Keep inputs/outputs typed and minimal so LLMs can call them safely.
4. **Advertise capabilities.** In `initialize` return your tool/resource metadata so Cursor/Claude Desktop lists them automatically.
5. **Run & register.** Start the server (e.g., `python mcp_server.py`) and add that command under `Cursor → Settings → MCP Servers`.

### DuckDB FastMCP example

Here's a minimal Python scaffolding for DuckDB:

```python
from fastmcp import FastMCP
import duckdb

con = duckdb.connect('retail.duckdb')
mcp = FastMCP()

@mcp.tool(name="duckdb_describe_customer")
def describe_customer(limit: int = 5) -> list[dict[str, str]]:
    """Return sample customer rows from DuckDB."""
    rows = con.execute(
        "SELECT c_customer_sk, c_first_name, c_last_name FROM customer LIMIT ?",
        [limit],
    ).fetchall()
    return [dict(row) for row in rows]


if __name__ == "__main__":
    mcp.run()
```

Register `python mcp_server.py` as a custom MCP server and the `describe_customer` tool becomes available directly in your prompts.

> **Shortcut:** Don’t want to build your own? MotherDuck ships an OSS MCP server that connects to both DuckDB and MotherDuck backends, complete with SaaS/read-only modes and Claude/Cursor examples. Install it via `uvx mcp-server-motherduck …` using the instructions in their repo: https://github.com/motherduckdb/mcp-server-motherduck.

### LanceDB FastMCP example

LanceDB also provides an MCP server that exposes read and write tools. Here's a minimal example.

```python
from mcp.server.fastmcp import FastMCPServer
import lancedb

# Re-use the `lance_db` directory populated earlier
# (or point to any LanceDB-backed dataset you want to expose)
db = lancedb.connect("lance_db")
server = FastMCPServer()

@mcp.tool(name="lancedb_search_logs")
def search_logs(query: str, limit: int = 5) -> list[dict]:
    """Return log rows semantically similar to the text query."""
    table = db.open_table("logs")
    matches = table.search(query).limit(limit).to_list()
    return matches

@server.tool(name="lancedb_list_tables")
def list_tables() -> list[str]:
    """List available LanceDB tables."""
    return db.table_names()

if __name__ == "__main__":
    server.run()
```

Register this script the same way as the DuckDB example and you now have semantic search tools on tap. Prefer a maintained implementation instead? The LanceDB team ships a ready-to-run FastMCP server (CLI plus config examples) here: https://github.com/lancedb/lancedb-mcp-server.
